In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
from pprint import pprint as pp
import csv
from pathlib import Path
import seaborn as sns
from itertools import product
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline 

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score, classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.metrics import homogeneity_score, silhouette_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import MiniBatchKMeans, DBSCAN
import fasttext

import gensim
from gensim import corpora

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

In [4]:
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from gensim.models import Word2Vec
from gensim.models import FastText
import numpy as np
np.random.seed(42)

In [5]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_predict

In [6]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek

In [7]:
pd.set_option('display.max_columns', None)

### Retrieving the data

#### DF 7

In [9]:
#df7 = pd.read_csv('df7.csv')
df8 = pd.read_csv('df8.csv', index_col=0)

In [10]:
df8.head(2)

,id,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,pymnt_plan,purpose,title,addr_state,earliest_cr_line,last_pymnt_d,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,acc_now_delinq,chargeoff_within_12_mths,tax_liens,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,zip_code,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,delinq_amnt,pub_rec_bankruptcies,target,content
0,1077501,36months,B,B2,Unknown,10,RENT,Verified,2011,False,credit_card,Computer,AZ,1985,2015,0,2016,False,False,False,False,C0Q1,C1Q1,C2Q2,C3Q3,C4Q2,C5Q0,C6Q7,C7Q9,C8Q0,C9Q7,C10Q7,C11Q0,C12Q3,C13Q0,C14Q0,C15Q0,C16Q6,C17Q8,C18Q0,C19Q0,C20Q0,C21Q2,C22Q3,C23Q2,C24Q3,C25Q0,C26Q0,C27Q0,C28Q1,C29Q7,C30Q7,C31Q0,C32Q0,__label__1,__label__1 1077501 36months B B2 Unknown 10 R...
1,1077430,60months,C,C4,Ryder,1,RENT,Source Verified,2011,False,car,bike,GA,1999,2013,0,2016,False,False,False,False,C0Q0,C1Q0,C2Q0,C3Q7,C4Q0,C5Q0,C6Q3,C7Q0,C8Q0,C9Q7,C10Q7,C11Q3,C12Q3,C13Q0,C14Q0,C15Q0,C16Q1,C17Q1,C18Q0,C19Q0,C20Q0,C21Q0,C22Q0,C23Q0,C24Q1,C25Q0,C26Q1,C27Q1,C28Q1,C29Q0,C30Q0,C31Q0,C32Q0,__label__2,__label__2 1077430 60months C C4 Ryder 1 REN...


# Models

In [11]:
# Initialize lists to store evaluation metrics for each fold
dataset_used = []
model_used = []
data_balancing_technique = []
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
conf_matrices = []

In [12]:
#combined_metrics = pd.DataFrame()

combined_metrics = pd.DataFrame(columns=['dataset', 'model', 'data balancing technique', 'fold', 'precision_1','precision_2','recall_1','recall_2','f1-score_1','f1-score_2','support_1','support_2','TP','FP','TN','FN'])

In [12]:
# Assuming 'df7' is your DataFrame with features and the target column
features = df7['content'].apply(lambda x: x.split(' ', 1)[1])  # Drop the target column to get the feature columns
target = df7['target'].apply(lambda x: int(x.split("__label__")[1]))  # Target column to predict

In [28]:
features = features.apply(preprocess)

In [13]:
stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [14]:
# Splitting the dataset into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [15]:
X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)

In [19]:
X_train.head(2)

40315    541691 6991370 122250 122250 122250 60months 0...
15335    751654 9511120 25250 25250 25250 60months 0179...
Name: content, dtype: object

In [20]:
sentences = [sentence.split() for sentence in X_train]
    
# Train the FastText model
fasttext_model = FastText(sentences, vector_size=100, window=20, min_count=2, workers=4, seed=42)

In [21]:
def vectorize(sentence):
        words = sentence.split()
        words_vecs = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
        if len(words_vecs) == 0:
            return np.zeros(100)
        words_vecs = np.array(words_vecs)
        return words_vecs.mean(axis=0)

In [23]:
X_train = np.array([vectorize(sentence) for sentence in X_train])
X_test = np.array([vectorize(sentence) for sentence in X_test])

In [24]:
clf = LinearRegression()
clf.fit(X_train, y_train)

LinearRegression()

In [25]:
y_pred = clf.predict(X_test)
    
y_pred = np.round(y_pred)
    
y_pred[y_pred <= 0] = 1
y_pred[y_pred >= 2] = 2

In [26]:
# Model evaluation
accuracy = accuracy_score(y_test, y_pred.round())
precision = precision_score(y_test, y_pred.round(), average='weighted')
recall = recall_score(y_test, y_pred.round(), average='weighted')
f1 = f1_score(y_test, y_pred.round(), average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred.round())

In [27]:
print(accuracy, precision, recall, f1, conf_matrix)

0.926766192547314 0.9249312897098703 0.926766192547314 0.9194314890353225 [[7194   80]
 [ 543  690]]


# Fast Text 
### Linear Regression

## DF7

In [13]:
# Assuming 'df7' is your DataFrame with features and the target column
features = df8['content'].apply(lambda x: x.split(' ', 1)[1])  # Drop the target column to get the feature columns
target = df8['target'].apply(lambda x: int(x.split("__label__")[1]))  # Target column to predict

stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

features = features.apply(preprocess)

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    #X_train = X_train.apply(preprocess)
    #X_test = X_test.apply(preprocess)

    sentences = [sentence.split() for sentence in X_train]
    
    # Train the FastText model
    fasttext_model = FastText(sentences, vector_size=100, window=20, min_count=2, workers=4, seed=42)
    
    def vectorize(sentence):
        words = sentence.split()
        words_vecs = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
        if len(words_vecs) == 0:
            return np.zeros(100)
        words_vecs = np.array(words_vecs)
        return words_vecs.mean(axis=0)

    X_train = np.array([vectorize(sentence) for sentence in X_train])
    X_test = np.array([vectorize(sentence) for sentence in X_test])
    
    #clf = LogisticRegression()
    clf = LinearRegression()
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    y_pred = np.round(y_pred)
    
    y_pred[y_pred <= 0] = 1
    y_pred[y_pred >= 2] = 2
    
    # Model evaluation
    accuracy = accuracy_score(y_test, y_pred.round())
    precision = precision_score(y_test, y_pred.round(), average='weighted')
    recall = recall_score(y_test, y_pred.round(), average='weighted')
    f1 = f1_score(y_test, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred.round())
    
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF8',
    'model' : 'Fast Text - Linear Regression',
    'data balancing technique' : 'None',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)
    
    print(i)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

2
3
4
5
6
7
8
9
10
11
Mean Accuracy: 0.97
Mean Precision: 0.97
Mean Recall: 0.97
Mean F1-Score: 0.97


In [14]:
combined_metrics[(combined_metrics['data balancing technique'] == 'None') & (combined_metrics['dataset'] == 'DF7')]#.head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
0,DF7,Fast Text - Linear Regression,None,1,0.966543,0.993852,0.999177,0.793781,0.982589,0.882621,3643.0,611.0,3640,126,485,3
1,DF7,Fast Text - Linear Regression,None,2,0.970848,0.996117,0.999449,0.824759,0.984941,0.902375,3632.0,622.0,3630,109,513,2
2,DF7,Fast Text - Linear Regression,None,3,0.965517,0.996101,0.999447,0.798438,0.982189,0.886383,3614.0,640.0,3612,129,511,2
3,DF7,Fast Text - Linear Regression,None,4,0.964696,0.994175,0.999169,0.795031,0.981630,0.883520,3610.0,644.0,3607,132,512,3
4,DF7,Fast Text - Linear Regression,None,5,0.968842,0.988701,0.998339,0.819033,0.983370,0.895904,3613.0,641.0,3607,116,525,6
5,DF7,Fast Text - Linear Regression,None,6,0.975128,0.989170,0.998339,0.856250,0.986597,0.917923,3613.0,640.0,3607,92,548,6
6,DF7,Fast Text - Linear Regression,None,7,0.965823,0.992551,0.998887,0.807576,0.982077,0.890560,3593.0,660.0,3589,127,533,4
7,DF7,Fast Text - Linear Regression,None,8,0.964931,0.989011,0.998325,0.805970,0.981344,0.888158,3583.0,670.0,3577,130,540,6
8,DF7,Fast Text - Linear Regression,None,9,0.964247,0.992495,0.998886,0.799094,0.981261,0.885356,3591.0,662.0,3587,133,529,4
9,DF7,Fast Text - Linear Regression,None,10,0.969884,0.992509,0.998892,0.825545,0.984175,0.901361,3611.0,642.0,3607,112,530,4


### FastText Linear Regression with df7 data and imbalance data tackling (RandomUnderSampler)

In [15]:
# Assuming 'df7' is your DataFrame with features and the target column
features = df8['content'].apply(lambda x: x.split(' ', 1)[1])  # Drop the target column to get the feature columns
target = df8['target'].apply(lambda x: int(x.split("__label__")[1]))  # Target column to predict

stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

features = features.apply(preprocess)

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    #X_train = X_train.apply(preprocess)
    #X_test = X_test.apply(preprocess)

    sentences = [sentence.split() for sentence in X_train]
    
    # Train the FastText model
    fasttext_model = FastText(sentences, vector_size=100, window=20, min_count=2, workers=4, seed=42)
    
    def vectorize(sentence):
        words = sentence.split()
        words_vecs = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
        if len(words_vecs) == 0:
            return np.zeros(100)
        words_vecs = np.array(words_vecs)
        return words_vecs.mean(axis=0)

    X_train = np.array([vectorize(sentence) for sentence in X_train])
    X_test = np.array([vectorize(sentence) for sentence in X_test])
    
    # Apply random undersampling
    rus = RandomUnderSampler(random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
    
    #clf = LogisticRegression()
    clf = LinearRegression()
    clf.fit(X_train_resampled, y_train_resampled)
    
    # Apply random undersampling to test data
    X_test_resampled, y_test_resampled = rus.fit_resample(X_test, y_test)
    
    y_pred = clf.predict(X_test_resampled)
    
    y_pred = np.round(y_pred)
    
    y_pred[y_pred <= 0] = 1
    y_pred[y_pred >= 2] = 2
    
    # Model evaluation
    accuracy = accuracy_score(y_test_resampled, y_pred.round())
    precision = precision_score(y_test_resampled, y_pred.round(), average='weighted')
    recall = recall_score(y_test_resampled, y_pred.round(), average='weighted')
    f1 = f1_score(y_test_resampled, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test_resampled, y_pred.round())
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test_resampled, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test_resampled, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF8',
    'model' : 'Fast Text - Linear Regression',
    'data balancing technique' : 'Random Under Sampler',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)
    
    print(i)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

2
3
4
5
6
7
8
9
10
11
Mean Accuracy: 0.96
Mean Precision: 0.96
Mean Recall: 0.96
Mean F1-Score: 0.96


In [16]:
combined_metrics[(combined_metrics['data balancing technique'] == 'Random Under Sampler') & (combined_metrics['dataset'] == 'DF8')].head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
10,DF8,Fast Text - Linear Regression,Random Under Sampler,1,0.917051,0.975482,0.977087,0.911620,0.946117,0.942470,611.0,611.0,597,54,557,14
11,DF8,Fast Text - Linear Regression,Random Under Sampler,2,0.941628,0.984823,0.985531,0.938907,0.963079,0.961317,622.0,622.0,613,38,584,9


### FastText Linear Regression with df7 data and imbalance data tackling (RandomOverSampler)

In [17]:
# Assuming 'df7' is your DataFrame with features and the target column
features = df8['content'].apply(lambda x: x.split(' ', 1)[1])  # Drop the target column to get the feature columns
target = df8['target'].apply(lambda x: int(x.split("__label__")[1]))  # Target column to predict

stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

features = features.apply(preprocess)

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    #X_train = X_train.apply(preprocess)
    #X_test = X_test.apply(preprocess)

    sentences = [sentence.split() for sentence in X_train]
    
    # Train the FastText model
    fasttext_model = FastText(sentences, vector_size=100, window=20, min_count=2, workers=4, seed=42)
    
    def vectorize(sentence):
        words = sentence.split()
        words_vecs = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
        if len(words_vecs) == 0:
            return np.zeros(100)
        words_vecs = np.array(words_vecs)
        return words_vecs.mean(axis=0)

    X_train = np.array([vectorize(sentence) for sentence in X_train])
    X_test = np.array([vectorize(sentence) for sentence in X_test])
    
    # Apply random undersampling
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
    
    #clf = LogisticRegression()
    clf = LinearRegression()
    clf.fit(X_train_resampled, y_train_resampled)
    
    # Apply random undersampling to test data
    X_test_resampled, y_test_resampled = rus.fit_resample(X_test, y_test)
    
    y_pred = clf.predict(X_test_resampled)
    
    y_pred = np.round(y_pred)
    
    y_pred[y_pred <= 0] = 1
    y_pred[y_pred >= 2] = 2
    
    # Model evaluation
    accuracy = accuracy_score(y_test_resampled, y_pred.round())
    precision = precision_score(y_test_resampled, y_pred.round(), average='weighted')
    recall = recall_score(y_test_resampled, y_pred.round(), average='weighted')
    f1 = f1_score(y_test_resampled, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test_resampled, y_pred.round())
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test_resampled, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test_resampled, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF7',
    'model' : 'Fast Text - Linear Regression',
    'data balancing technique' : 'Random Over Sampler',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)
    
    print(i)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

2
3
4
5
6
7
8
9
10
11
Mean Accuracy: 0.96
Mean Precision: 0.96
Mean Recall: 0.96
Mean F1-Score: 0.96


In [18]:
combined_metrics[(combined_metrics['data balancing technique'] == 'Random Over Sampler') & (combined_metrics['dataset'] == 'DF7')]#.head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
20,DF7,Fast Text - Linear Regression,Random Over Sampler,1,0.920437,0.963855,0.965630,0.916530,0.942492,0.939597,611.0,611.0,590,51,560,21
21,DF7,Fast Text - Linear Regression,Random Over Sampler,2,0.945312,0.971854,0.972669,0.943730,0.958796,0.957586,622.0,622.0,605,35,587,17
22,DF7,Fast Text - Linear Regression,Random Over Sampler,3,0.935917,0.980296,0.981250,0.932813,0.958047,0.955965,640.0,640.0,628,43,597,12
23,DF7,Fast Text - Linear Regression,Random Over Sampler,4,0.919708,0.976783,0.978261,0.914596,0.948081,0.944667,644.0,644.0,630,55,589,14
24,DF7,Fast Text - Linear Regression,Random Over Sampler,5,0.945946,0.982143,0.982839,0.943838,0.964040,0.962609,641.0,641.0,630,36,605,11
25,DF7,Fast Text - Linear Regression,Random Over Sampler,6,0.948092,0.969600,0.970313,0.946875,0.959073,0.958103,640.0,640.0,621,34,606,19
26,DF7,Fast Text - Linear Regression,Random Over Sampler,7,0.931232,0.983923,0.984848,0.927273,0.957290,0.954758,660.0,660.0,650,48,612,10
27,DF7,Fast Text - Linear Regression,Random Over Sampler,8,0.924394,0.965571,0.967164,0.920896,0.945295,0.942704,670.0,670.0,648,53,617,22
28,DF7,Fast Text - Linear Regression,Random Over Sampler,9,0.924747,0.963665,0.965257,0.921450,0.944568,0.942085,662.0,662.0,639,52,610,23
29,DF7,Fast Text - Linear Regression,Random Over Sampler,10,0.919881,0.963934,0.965732,0.915888,0.942249,0.939297,642.0,642.0,620,54,588,22


In [20]:
# Replace values 'DF7' with 'DF8' in the 'dataset' column
combined_metrics['dataset'] = combined_metrics['dataset'].replace('DF7', 'DF8')

## Performance metrics for the Linear Regression

In [22]:
combined_metrics

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
0,DF8,Fast Text - Linear Regression,None,1,0.966543,0.993852,0.999177,0.793781,0.982589,0.882621,3643.0,611.0,3640,126,485,3
1,DF8,Fast Text - Linear Regression,None,2,0.970848,0.996117,0.999449,0.824759,0.984941,0.902375,3632.0,622.0,3630,109,513,2
2,DF8,Fast Text - Linear Regression,None,3,0.965517,0.996101,0.999447,0.798438,0.982189,0.886383,3614.0,640.0,3612,129,511,2
3,DF8,Fast Text - Linear Regression,None,4,0.964696,0.994175,0.999169,0.795031,0.981630,0.883520,3610.0,644.0,3607,132,512,3
4,DF8,Fast Text - Linear Regression,None,5,0.968842,0.988701,0.998339,0.819033,0.983370,0.895904,3613.0,641.0,3607,116,525,6
5,DF8,Fast Text - Linear Regression,None,6,0.975128,0.989170,0.998339,0.856250,0.986597,0.917923,3613.0,640.0,3607,92,548,6
6,DF8,Fast Text - Linear Regression,None,7,0.965823,0.992551,0.998887,0.807576,0.982077,0.890560,3593.0,660.0,3589,127,533,4
7,DF8,Fast Text - Linear Regression,None,8,0.964931,0.989011,0.998325,0.805970,0.981344,0.888158,3583.0,670.0,3577,130,540,6
8,DF8,Fast Text - Linear Regression,None,9,0.964247,0.992495,0.998886,0.799094,0.981261,0.885356,3591.0,662.0,3587,133,529,4
9,DF8,Fast Text - Linear Regression,None,10,0.969884,0.992509,0.998892,0.825545,0.984175,0.901361,3611.0,642.0,3607,112,530,4


In [23]:
# Assuming df is your DataFrame
combined_metrics.to_csv('Output Data/Fast Text - Linear Regression2.csv')